In [1]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

from load_data import (load_tourism,
                       create_train_dataloader,
                       create_backtest_dataloader,
                       create_test_dataloader)
from sur_ts import SurrogateTimeSeriesTransformerConfig, SurrogateTimeSeriesTransformer


/Users/panda/anaconda3/envs/ts/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
freq = '1M'
train_dataset, test_dataset, lags_sequence, time_features = load_tourism()
prediction_length = 24


config = SurrogateTimeSeriesTransformerConfig(
    prediction_length=prediction_length,
    # context length:
    context_length=prediction_length * 2,
    # lags coming from helper given the freq:
    lags_sequence=lags_sequence,
    # we'll add 2 time features ("month of year" and "age", see further):
    num_time_features=len(time_features) + 1,
    # we have a single static categorical feature, namely time series ID:
    num_static_categorical_features=1,
    # it has 366 possible values:
    cardinality=[len(train_dataset)],
    # the model will learn an embedding of size 2 for each of the 366 possible values:
    embedding_dimension=[2],
    
    # transformer params:
    encoder_layers=4,
    decoder_layers=4,
    d_model=32,
)

In [3]:
train_dataloader = create_train_dataloader(
    config=config,
    freq=freq,
    data=train_dataset,
    batch_size=256,
    num_batches_per_epoch=100,
)

test_dataloader = create_backtest_dataloader(
    config=config,
    freq=freq,
    data=test_dataset,
    batch_size=64,
)

In [4]:
sur_model = SurrogateTimeSeriesTransformer(config)
batch = next(iter(train_dataloader))
sur_outputs = sur_model(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"]
    if config.num_static_categorical_features > 0
    else None,
    static_real_features=batch["static_real_features"]
    if config.num_static_real_features > 0
    else None,
    future_values=batch["future_values"],
    future_time_features=batch["future_time_features"],
    future_observed_mask=batch["future_observed_mask"],
    output_hidden_states=True,
)
print("attention_output shape: " ,sur_outputs.attention_output.shape)
print("attention_weights shape: " ,sur_outputs.attention_weights.shape)
print("last_encoder_state shape: " ,sur_outputs.encoder_hidden_states[-1].shape)
print("last_decoder_state shape: " ,sur_outputs.decoder_hidden_states[-1].shape)
print("Surrogate loss: ", sur_outputs.sur_loss)
print("Prediction loss: ", sur_outputs.pred_loss)
print("Total loss: ", sur_outputs.loss)

attention_output shape:  torch.Size([256, 24, 32])
attention_weights shape:  torch.Size([256, 24, 85])
last_encoder_state shape:  torch.Size([256, 48, 32])
last_decoder_state shape:  torch.Size([256, 24, 32])
Surrogate loss:  tensor(8.6111, grad_fn=<DivBackward0>)
Prediction loss:  tensor(9.0177, grad_fn=<DivBackward0>)
Total loss:  tensor(17.6288, grad_fn=<AddBackward0>)


In [5]:
# # test how many batch per epoch
# for idx, batch in enumerate(train_dataloader):
#     print(idx)

In [6]:
from accelerate import Accelerator
from torch.optim import AdamW
import torch
from transformers import AutoModel

accelerator = Accelerator()
device = accelerator.device
# device = 'cpu'

sur_model.to(device)
optimizer = AdamW(sur_model.parameters(), lr=6e-4, betas=(0.9, 0.95), weight_decay=1e-1)

sur_model, optimizer, train_dataloader = accelerator.prepare(
    sur_model,
    optimizer,
    train_dataloader,
)

sur_model.train()

print(f"Started training: ")
for epoch in range(40):
    for idx, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = sur_model(
            static_categorical_features=batch["static_categorical_features"].to(device)
            if config.num_static_categorical_features > 0
            else None,
            static_real_features=batch["static_real_features"].to(device)
            if config.num_static_real_features > 0
            else None,
            past_time_features=batch["past_time_features"].to(device),
            past_values=batch["past_values"].to(device),
            future_time_features=batch["future_time_features"].to(device),
            future_values=batch["future_values"].to(device),
            past_observed_mask=batch["past_observed_mask"].to(device),
            future_observed_mask=batch["future_observed_mask"].to(device),
            output_hidden_states=True,
        )
        loss = outputs.loss

        # Backpropagation
        accelerator.backward(loss)
        optimizer.step()

        if idx % 100 == 0:
            pred_loss = outputs.pred_loss
            sur_loss = outputs.sur_loss
            sim_loss = outputs.sim_loss
            print(f"Epoch [{epoch+1}] Step [{idx+1}]: loss: {loss.item():.4f},\tpred loss: "
                  f"{pred_loss.item():.4f},\tsur loss: {sur_loss.item():.4f},\tsim loss: {sim_loss.item():.4f}")  


Started training: 
Epoch [1] Step [1]: loss: 17.4993,	pred loss: 8.9393,	sur loss: 8.5600
Epoch [2] Step [1]: loss: 16.2949,	pred loss: 8.0233,	sur loss: 8.2716
Epoch [3] Step [1]: loss: 14.8077,	pred loss: 7.1944,	sur loss: 7.6133
Epoch [4] Step [1]: loss: 15.8590,	pred loss: 7.5203,	sur loss: 8.3388
Epoch [5] Step [1]: loss: 16.3298,	pred loss: 7.7318,	sur loss: 8.5981
Epoch [6] Step [1]: loss: 15.1802,	pred loss: 7.2889,	sur loss: 7.8912
Epoch [7] Step [1]: loss: 14.0745,	pred loss: 6.7984,	sur loss: 7.2760
Epoch [8] Step [1]: loss: 14.4217,	pred loss: 6.9961,	sur loss: 7.4256
Epoch [9] Step [1]: loss: 16.2568,	pred loss: 7.7926,	sur loss: 8.4643
Epoch [10] Step [1]: loss: 14.9980,	pred loss: 7.2231,	sur loss: 7.7749
Epoch [11] Step [1]: loss: 14.3482,	pred loss: 6.8318,	sur loss: 7.5163
Epoch [12] Step [1]: loss: 14.2794,	pred loss: 6.9355,	sur loss: 7.3439
Epoch [13] Step [1]: loss: 15.4631,	pred loss: 7.4835,	sur loss: 7.9796
Epoch [14] Step [1]: loss: 14.1998,	pred loss: 6.9530,

KeyboardInterrupt: 

In [7]:
# Specify the file path to save the model
model_path = "model2"

# Save the model
sur_model.save_pretrained(model_path)

In [ ]:
import numpy as np
sur_model.eval()

forecasts = []

for batch in test_dataloader:
    outputs = sur_model.generate(
        static_categorical_features=batch["static_categorical_features"].to(device)
        if config.num_static_categorical_features > 0
        else None,
        static_real_features=batch["static_real_features"].to(device)
        if config.num_static_real_features > 0
        else None,
        past_time_features=batch["past_time_features"].to(device),
        past_values=batch["past_values"].to(device),
        future_time_features=batch["future_time_features"].to(device),
        past_observed_mask=batch["past_observed_mask"].to(device),
        # future_observed_mask=None,
    )
    forecasts.append(outputs.sequences.cpu().numpy())
forecasts = np.vstack(forecasts)
print(forecasts.shape)

/Users/panda/anaconda3/envs/ts/lib/python3.11/site-packages/torch/distributions/gamma.py:12: UserWarning: The operator 'aten::_standard_gamma' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  return torch._standard_gamma(concentration)


(366, 100, 24)


In [ ]:
from evaluate import load
from gluonts.time_feature import get_seasonality

mase_metric = load("evaluate-metric/mase")
smape_metric = load("evaluate-metric/smape")

forecast_median = np.median(forecasts, 1)

mase_metrics = []
smape_metrics = []
for item_id, ts in enumerate(test_dataset):
    training_data = ts["target"][:-prediction_length]
    ground_truth = ts["target"][-prediction_length:]
    mase = mase_metric.compute(
        predictions=forecast_median[item_id], 
        references=np.array(ground_truth), 
        training=np.array(training_data), 
        periodicity=get_seasonality(freq))
    mase_metrics.append(mase["mase"])
    
    smape = smape_metric.compute(
        predictions=forecast_median[item_id], 
        references=np.array(ground_truth), 
    )
    smape_metrics.append(smape["smape"])

print(f"MASE: {np.mean(mase_metrics)}")
print(f"sMAPE: {np.mean(smape_metrics)}")

MASE: 1.879967461266539
sMAPE: 0.2390452388073287
